<a href="https://colab.research.google.com/github/boneeyah/SMU/blob/main/Dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
!pip install jupyter-dash
!pip install dash-bootstrap-components

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [112]:
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd

In [113]:
from pandas.io.parsers.readers import read_csv
df = read_csv("https://raw.githubusercontent.com/boneeyah/SMU/main/netflix_titles.csv")

In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [115]:
df = df.drop(index=df[df['country'].isnull()].index)

In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7976 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7976 non-null   object
 1   type          7976 non-null   object
 2   title         7976 non-null   object
 3   director      5751 non-null   object
 4   cast          7305 non-null   object
 5   country       7976 non-null   object
 6   date_added    7967 non-null   object
 7   release_year  7976 non-null   int64 
 8   rating        7973 non-null   object
 9   duration      7973 non-null   object
 10  listed_in     7976 non-null   object
 11  description   7976 non-null   object
dtypes: int64(1), object(11)
memory usage: 810.1+ KB


In [152]:
colors = {
    'background':'#221f1f',
    'text':'#7fdbff'
}

In [134]:
df1 = df[['country','type','show_id']]

df1 = df1[['country','show_id']].groupby(by='country',as_index=False).count()
fig1 = px.scatter_geo(data_frame=df1,locations='country',size='show_id',locationmode='country names',width = 900)
fig1.update_layout(plot_bgcolor=colors['background'],paper_bgcolor=colors['background'],font_color = 'white')
fig1.update_geos(bgcolor=colors['background'],landcolor='#747A76',lakecolor='black',showcountries=True)

In [139]:
df2 = df[['type','country','show_id','release_year']].groupby(by=['release_year','type'],as_index=False).count()
fig2 = px.bar(data_frame=df2,x='release_year',y='show_id',color='type',width = 900)
fig2.update_layout(plot_bgcolor=colors['background'],paper_bgcolor=colors['background'],font_color = 'white')

In [ ]:
df3 = df[['rating','type','show_id']]
df3 = df3.groupby(['rating','type'],as_index=False).count()
fig3 = px.treemap(data_frame=df3,path=[px.Constant('Rating'),'type','rating'],values='show_id',color_discrete_map={'(?)':'#747A76','Movie':'blue','TV Show':'red'},width = 1800)
fig3.update_layout(paper_bgcolor=colors['background'],font_color='white')

In [ ]:
app = JupyterDash(__name__)
app.layout = html.Div(style = {'backgroundColor':colors['background']},children = [
    html.H1('Netflix Catalog - Movies and Shows',style = {'color':colors['text']}),
    html.Div([
      html.Div([dcc.Graph(id='graph2',figure=fig1)],style={'display':'inline-block'}),
      html.Div([dcc.Graph(id='graph3',figure=fig2)],style={'display':'inline-block'})
    ]),
    html.Div([dcc.Graph(id='graph4',figure=fig3)],style={'display':'inline-block'})
    ])


if __name__ == '__main__':
  app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>

In [132]:
df[['country','type','show_id']].groupby(by='country',as_index=False).count()

,country,type,show_id
0,", France, Algeria",1,1
1,", South Korea",1,1
2,Argentina,56,56
3,"Argentina, Brazil, France, Poland, Germany, De...",1,1
4,"Argentina, Chile",2,2
...,...,...,...
743,Venezuela,1,1
744,"Venezuela, Colombia",1,1
745,Vietnam,7,7
746,West Germany,1,1


Testing dropdown

In [189]:
external_stylesheets = ['https://raw.githubusercontent.com/boneeyah/SMU/main/dropdown.css']

In [216]:
app = JupyterDash(__name__)
app.layout = html.Div(children = [
    html.H1('Netflix Catalog - Movies and Shows',style={'font-family':'Helvetica','font-size':'24px','color':'#e50914'}),
    html.Div([
        dcc.Dropdown(
            options=['Both','TV Show','Movie'],
            id='content-filter',
            value='Both',
            clearable = False,
            style={'width':'40%',
                   'display':'inline-block',
                   'margin-left':'4%'}
        ),
        html.Div([dcc.RangeSlider(
            id='content-year',
            min=df['release_year'].min(),
            max=df['release_year'].max(),
            marks={1942:'1942',
                   1950:'1950',
                   1960:'1960',
                   1970:'1970',
                   1980:'1980',
                   1990:'1990',
                   2000:'2000',
                   2010:'2010',
                   2021:'2021'},
            value=[df['release_year'].min(),df['release_year'].max()]
        )],
        style={'width':'50%',
               'display':'inline-block',
               'margin-left':'3%'})
        
    ]),
    html.Div([
      html.Div([
          dcc.Graph(id='graph1')
          ],
          style={'display':'inline-block',
                 'width':'49%'}),
      html.Div([
          dcc.Graph(id='graph2')],
          style={'display':'inline-block',
                 'margin-left':'.25%',
                 'width':'49%'}
          )
    ]),
    html.Div([
        dcc.Graph(id='graph3')],
        style={'display':'inline-block',
               'width':'98.25%'})
    ])
@app.callback(
    Output('graph1','figure'),
    Input('content-filter','value'),
    Input('content-year','value')
)
def create_fig1(selected_type,selected_years):
  if selected_type=='Both':
    dff=df[df.release_year.between(selected_years[0],selected_years[1])]
  else:
    dff=df[(df.type==selected_type) & (df.release_year.between(selected_years[0],selected_years[1]))]
  dff1=dff[['country','type','show_id']].groupby(by='country',as_index=False).count()
  dff1=dff1.rename(columns={'show_id':'Count'})
  fig = px.scatter_geo(data_frame=dff1,
                       locations='country',
                       size='Count',
                       locationmode='country names',
                       height=440)
  fig.update_layout(plot_bgcolor=colors['background'],
                    paper_bgcolor=colors['background'],
                    font_color = 'white')
  fig.update_geos(bgcolor='#f5f5f1',
                  landcolor='#747A76',
                  lakecolor='black',
                  showcountries=True)
  fig.update_traces(marker_color='#e50914')
  return fig
@app.callback(
    Output('graph2','figure'),
    Input('content-filter','value'),
    Input('content-year','value')
)
def create_fig2(selected_type,selected_years):
  if selected_type=='Both':
    dff=df[df.release_year.between(selected_years[0],selected_years[1])]
  else:
    dff=df[(df.type==selected_type) & (df.release_year.between(selected_years[0],selected_years[1]))]
  #dff = df[df.type ==selected_type]
  dff2 = dff[['type','country','show_id','release_year']].groupby(by=['release_year','type'],as_index=False).count()
  fig = px.bar(data_frame=dff2,
               x='release_year',
               y='show_id',
               color='type',
               height=440,
               color_discrete_map={'(?)':'#221f1f','TV Show':'#f5f5f1','Movie':'#e50914'})
  fig.update_layout(plot_bgcolor=colors['background'],
                    paper_bgcolor=colors['background'],
                    font_color='white',yaxis_title='Count',
                    xaxis_title='Year')
  return fig
@app.callback(
    Output('graph3','figure'),
    Input('content-filter','value'),
    Input('content-year','value')
)
def create_fig3(selected_type,selected_years):
  if selected_type=='Both':
    dff=df[df.release_year.between(selected_years[0],selected_years[1])]
  else:
    dff=df[(df.type==selected_type) & (df.release_year.between(selected_years[0],selected_years[1]))]
  dff3 = dff[['rating','type','show_id']].groupby(by=['rating','type'],as_index=False).count()
  fig = px.treemap(data_frame=dff3,
                   path=[px.Constant('Rating'),'type','rating'],
                   color='type',values='show_id',
                   color_discrete_map={'(?)':'#221f1f',
                                       'TV Show':'#f5f5f1',
                                       'Movie':'#e50914'},
                   height=440)
  fig.update_layout(paper_bgcolor=colors['background'],font_color='white')
  return fig

if __name__ == '__main__':
  app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>